In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#export
from lib.nb_04 import *

- https://medium.com/huggingface/multi-label-text-classification-using-bert-the-mighty-transformer-69714fa3fb3d
- https://www.youtube.com/channel/UCXvHuBMbgJw67i5vrMBBobA
- https://arxiv.org/pdf/1904.10324.pdf
- https://www.youtube.com/watch?v=5vcj8kSwBCY&list=PLoROMvodv4rOhcuXMZkNm7j3fVwBBY42z&index=15&t=2416s
- https://ai.googleblog.com/2019/04/evaluating-unsupervised-learning-of.html
- https://www.youtube.com/watch?v=qzljG6DKgic&feature=youtu.be
- https://arxiv.org/pdf/1904.11272.pdf

### Images Recap

In [4]:
bs=64

In [5]:
path = datasets.untar_data(datasets.URLs.IMAGENETTE_160)
tfms = [MakeRGB(), ResizeFixed(128), to_byte_tensor, to_float_tensor]

il = ImageList.from_files(path, tfms=tfms)
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler, proc_y=CategoryProcessor())
data = ll.to_databunch(bs, c_in=3, c_out=10, num_workers=4)

In [6]:
data.train_ds

LabeledData
x: ImageList (12894 items)
[PosixPath('/Users/vks/.fastai/data/imagenette-160/train/n03394916/n03394916_58454.JPEG'), PosixPath('/Users/vks/.fastai/data/imagenette-160/train/n03394916/n03394916_32588.JPEG'), PosixPath('/Users/vks/.fastai/data/imagenette-160/train/n03394916/n03394916_32422.JPEG'), PosixPath('/Users/vks/.fastai/data/imagenette-160/train/n03394916/n03394916_33663.JPEG'), PosixPath('/Users/vks/.fastai/data/imagenette-160/train/n03394916/n03394916_27948.JPEG'), PosixPath('/Users/vks/.fastai/data/imagenette-160/train/n03394916/n03394916_38644.JPEG'), PosixPath('/Users/vks/.fastai/data/imagenette-160/train/n03394916/n03394916_35076.JPEG'), PosixPath('/Users/vks/.fastai/data/imagenette-160/train/n03394916/n03394916_38214.JPEG'), PosixPath('/Users/vks/.fastai/data/imagenette-160/train/n03394916/n03394916_23008.JPEG'), PosixPath('/Users/vks/.fastai/data/imagenette-160/train/n03394916/n03394916_33008.JPEG')...]
Path: /Users/vks/.fastai/data/imagenette-160
y: ItemList 

In [7]:
data.valid_ds

LabeledData
x: ImageList (500 items)
[PosixPath('/Users/vks/.fastai/data/imagenette-160/val/n03394916/ILSVRC2012_val_00046669.JPEG'), PosixPath('/Users/vks/.fastai/data/imagenette-160/val/n03394916/ILSVRC2012_val_00033682.JPEG'), PosixPath('/Users/vks/.fastai/data/imagenette-160/val/n03394916/ILSVRC2012_val_00005548.JPEG'), PosixPath('/Users/vks/.fastai/data/imagenette-160/val/n03394916/ILSVRC2012_val_00028723.JPEG'), PosixPath('/Users/vks/.fastai/data/imagenette-160/val/n03394916/ILSVRC2012_val_00048303.JPEG'), PosixPath('/Users/vks/.fastai/data/imagenette-160/val/n03394916/ILSVRC2012_val_00048138.JPEG'), PosixPath('/Users/vks/.fastai/data/imagenette-160/val/n03394916/ILSVRC2012_val_00025761.JPEG'), PosixPath('/Users/vks/.fastai/data/imagenette-160/val/n03394916/ILSVRC2012_val_00017407.JPEG'), PosixPath('/Users/vks/.fastai/data/imagenette-160/val/n03394916/ILSVRC2012_val_00035433.JPEG'), PosixPath('/Users/vks/.fastai/data/imagenette-160/val/n03394916/ILSVRC2012_val_00041247.JPEG')...]

## Text

### Text ItemList

In [8]:
path = datasets.untar_data(datasets.URLs.IMDB)

In [9]:
path.ls()

[PosixPath('/Users/vks/.fastai/data/imdb/ld.pkl'),
 PosixPath('/Users/vks/.fastai/data/imdb/test'),
 PosixPath('/Users/vks/.fastai/data/imdb/ll_lm.pkl'),
 PosixPath('/Users/vks/.fastai/data/imdb/tmp_clas'),
 PosixPath('/Users/vks/.fastai/data/imdb/vocab_lm.pkl'),
 PosixPath('/Users/vks/.fastai/data/imdb/imdb.vocab'),
 PosixPath('/Users/vks/.fastai/data/imdb/unsup'),
 PosixPath('/Users/vks/.fastai/data/imdb/README'),
 PosixPath('/Users/vks/.fastai/data/imdb/tmp_lm'),
 PosixPath('/Users/vks/.fastai/data/imdb/ll_clas.pkl'),
 PosixPath('/Users/vks/.fastai/data/imdb/train')]

In [10]:
#export
def read_file(fn): 
    with open(fn, 'r', encoding = 'utf8') as f: return f.read()
    
class TextList(ItemList):
    @classmethod
    def from_files(cls, path, extensions='.txt', recurse=True, include=None, **kwargs):
        return cls(get_files(path, extensions, recurse=recurse, include=include), path, **kwargs)
    
    def get(self, i):
        if isinstance(i, Path): return read_file(i)
        return i

In [11]:
il = TextList.from_files(path, include=['train', 'test', 'unsup'])

In [12]:
len(il.items)

100000

In [13]:
il

TextList (100000 items)
[PosixPath('/Users/vks/.fastai/data/imdb/test/neg/1821_4.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/9487_1.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/4604_4.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/2828_2.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/10890_1.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/3351_4.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/8070_2.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/1027_4.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/8248_3.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/4290_4.txt')...]
Path: /Users/vks/.fastai/data/imdb

In [14]:
txt = il[20]
txt

"Subspecies is set in Romania where two American college students Michele (Laura Mae Tate) & Lillian (Michelle McBride) arrive to study local folklore with the aid of local friend Mara (Irina Movila). There they rent rooms in a hotel & become curious about the mysterious ruins of a nearby castle, it turns out that a powerful & evil Vampire named Radu (Anders Hove) lives there who has stolen the Bloodstone from his father King Vladislav (Angus Scrimm). Radu takes a fancy to the three girls & starts drinking the blood of Mara & Lillian, meanwhile Michele falls for a guy named Stefan (Michael Watson) who just so happens to be Radu's brother. Michele & Stefan decide to team up & rid the world of the evil Radu...<br /><br />Directed by Ted Nicolaou this film seems to be quite highly regarded amongst genre fans & while it's not terrible I certainly wouldn't call it very good & I could't really see anything much to get excited about. Subspecies is a rather slow going film, not that much actua

In [15]:
il.items[20]

PosixPath('/Users/vks/.fastai/data/imdb/test/neg/2351_4.txt')

In [16]:
#export
import random

In [17]:
#export
def random_splitter(item, p_valid=0.1):
    return random.random() < p_valid

In [18]:
il

TextList (100000 items)
[PosixPath('/Users/vks/.fastai/data/imdb/test/neg/1821_4.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/9487_1.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/4604_4.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/2828_2.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/10890_1.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/3351_4.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/8070_2.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/1027_4.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/8248_3.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/4290_4.txt')...]
Path: /Users/vks/.fastai/data/imdb

In [19]:
sd = SplitData.split_by_func(il,partial(random_splitter, p_valid=0.1))

In [20]:
sd

SplitData
Train: TextList (90043 items)
[PosixPath('/Users/vks/.fastai/data/imdb/test/neg/1821_4.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/9487_1.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/4604_4.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/2828_2.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/10890_1.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/3351_4.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/8070_2.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/1027_4.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/4290_4.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/11890_1.txt')...]
Path: /Users/vks/.fastai/data/imdb
Valid: TextList (9957 items)
[PosixPath('/Users/vks/.fastai/data/imdb/test/neg/8248_3.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/10096_1.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/2008_1.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/1789_2.txt'), PosixP

In [21]:
sd.train[1]

'I have seen this movie and I did not care for this movie anyhow. I would not think about going to Paris because I do not like this country and its national capital. I do not like to learn french anyhow because I do not understand their language. Why would I go to France when I rather go to Germany or the United Kingdom? Germany and the United Kingdom are the nations I tolerate. Apparently the Olsen Twins do not understand the French language just like me. Therefore I will not bother the France trip no matter what. I might as well stick to the United Kingdom and meet single women and play video games if there is a video arcade. That is all.'

In [22]:
sd.valid[1]

"Worst movie ever seen. Worst acting too. I cannot imagine a movie worse then this. Nothing to see. No acting at all.T hey (actors) should look for another job. I cant't understand who was stupid enough to actually put money into this movie.<br /><br />I'm sorry for Eric Roberts. Must be tough...I cannot imagine how HUUUGE his mortgage must be to justify taking the job!<br /><br />The ladies in the movie...perhaps they better stick to XXX.<br /><br />As for the LEADING MAN...what a lead! He better be put on a lead and stay there! I can see him being more successful at barking rather than acting. <br /><br />Overall rating: Do NOt rent...DO NOT BUY!"

### Tokenization

We need to tokenize the dataset first, which is splitting a sentence in individual tokens. Those tokens are the basic words or punctuation signs with a few tweaks: don't for instance is split between do and n't. We will use a processor for this, in conjunction with the spacy library.

In [23]:
#export
import spacy,html

Before even tokenizeing, we will apply a bit of preprocessing on the texts to clean them up (we saw the one up there had some HTML code). These rules are applied before we split the sentences in tokens.

In [24]:
#export
#special tokens
UNK, PAD, BOS, EOS, TK_REP, TK_WREP, TK_UP, TK_MAJ = "xxunk xxpad xxbos xxeos xxrep xxwrep xxup xxmaj".split()

def sub_br(t):
    "Replaces the <br /> by \n"
    re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
    return re_br.sub("\n", t)

def spec_add_spaces(t):
    "Add spaces around / and #"
    return re.sub(r'([/#])', r' \1 ', t)

def rm_useless_spaces(t):
    "Remove multiple spaces"
    return re.sub(' {2,}', ' ', t)

def replace_rep(t):
    "Replace repetitions at the character level: cccc -> TK_REP 4 c"
    def _replace_rep(m:Collection[str]) -> str:
        c,cc = m.groups()
        return f' {TK_REP} {len(cc)+1} {c} '
    re_rep = re.compile(r'(\S)(\1{3,})')
    return re_rep.sub(_replace_rep, t)
    
def replace_wrep(t):
    "Replace word repetitions: word word word -> TK_WREP 3 word"
    def _replace_wrep(m:Collection[str]) -> str:
        c,cc = m.groups()
        return f' {TK_WREP} {len(cc.split())+1} {c} '
    re_wrep = re.compile(r'(\b\w+\W+)(\1{3,})')
    return re_wrep.sub(_replace_wrep, t)

def fixup_text(x):
    "Various messy things we've seen in documents"
    re1 = re.compile(r'  +')
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>',UNK).replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))
    
default_pre_rules = [fixup_text, replace_rep, replace_wrep, spec_add_spaces, rm_useless_spaces, sub_br]
default_spec_tok = [UNK, PAD, BOS, EOS, TK_REP, TK_WREP, TK_UP, TK_MAJ]

In [25]:
replace_rep('cccc')

' xxrep 4 c '

In [26]:
replace_wrep('word word word word word ')

' xxwrep 5 word  '

These rules are applies after the tokenization on the list of tokens.

In [27]:
#export
def replace_all_caps(x):
    "Replace tokens in ALL CAPS by their lower version and add `TK_UP` before."
    res = []
    for t in x:
        if t.isupper() and len(t) > 1: res.append(TK_UP); res.append(t.lower())
        else: res.append(t)
    return res

def deal_caps(x):
    "Replace all Capitalized tokens in by their lower version and add `TK_MAJ` before."
    res = []
    for t in x:
        if t == '': continue
        if t[0].isupper() and len(t) > 1 and t[1:].islower(): res.append(TK_MAJ)
        res.append(t.lower())
    return res

def add_eos_bos(x): return [BOS] + x + [EOS]

default_post_rules = [deal_caps, replace_all_caps, add_eos_bos]

In [28]:
replace_all_caps(['I', 'AM', 'SHOUTING'])

['I', 'xxup', 'am', 'xxup', 'shouting']

In [29]:
deal_caps(['My', 'name', 'is', 'Jeremy'])

['xxmaj', 'my', 'name', 'is', 'xxmaj', 'jeremy']

In [30]:
#export
from spacy.symbols import ORTH
from concurrent.futures import ProcessPoolExecutor

def parallel(func, arr, max_workers=4):
    if max_workers<2: results = list(progress_bar(map(func, enumerate(arr)), total=len(arr)))
    else:
        with ProcessPoolExecutor(max_workers=max_workers) as ex:
            return list(progress_bar(ex.map(func, enumerate(arr)), total=len(arr)))
    if any([o is not None for o in results]): return results

In [31]:
#export
class TokenizeProcessor(Processor):
    def __init__(self, lang="en", chunksize=2000, pre_rules=None, post_rules=None, max_workers=4): 
        self.chunksize,self.max_workers = chunksize,max_workers
        self.tokenizer = spacy.blank(lang).tokenizer
        for w in default_spec_tok:
            self.tokenizer.add_special_case(w, [{ORTH: w}])
        self.pre_rules  = default_pre_rules  if pre_rules  is None else pre_rules
        self.post_rules = default_post_rules if post_rules is None else post_rules

    def proc_chunk(self, args):
        i,chunk = args
        chunk = [compose(t, self.pre_rules) for t in chunk]
        docs = [[d.text for d in doc] for doc in self.tokenizer.pipe(chunk)]
        docs = [compose(t, self.post_rules) for t in docs]
        return docs

    def __call__(self, items): 
        toks = []
        if isinstance(items[0], Path): items = [read_file(i) for i in items]
        chunks = [items[i: i+self.chunksize] for i in (range(0, len(items), self.chunksize))]
        toks = parallel(self.proc_chunk, chunks, max_workers=self.max_workers)
        return sum(toks, [])
    
    def proc1(self, item): return self.proc_chunk([toks])[0]
    
    def deprocess(self, toks): return [self.deproc1(tok) for tok in toks]
    def deproc1(self, tok):    return " ".join(tok)

In [32]:
tp = TokenizeProcessor()

In [33]:
txt[:250]

'Subspecies is set in Romania where two American college students Michele (Laura Mae Tate) & Lillian (Michelle McBride) arrive to study local folklore with the aid of local friend Mara (Irina Movila). There they rent rooms in a hotel & become curious '

In [34]:
sd.train[0]

"Alan Rickman & Emma Thompson give good performances with southern/New Orleans accents in this detective flick. It's worth seeing for their scenes- and Rickman's scene with Hal Holbrook. These three actors mannage to entertain us no matter what the movie, it seems. The plot for the movie shows potential, but one gets the impression in watching the film that it was not pulled off as well as it could have been. The fact that it is cluttered by a rather uninteresting subplot and mostly uninteresting kidnappers really muddles things. The movie is worth a view- if for nothing more than entertaining performances by Rickman, Thompson, and Holbrook."

In [35]:
tp(sd.train[:3])

[['xxbos',
  'xxmaj',
  'alan',
  'xxmaj',
  'rickman',
  '&',
  'xxmaj',
  'emma',
  'xxmaj',
  'thompson',
  'give',
  'good',
  'performances',
  'with',
  'southern',
  '/',
  'xxmaj',
  'new',
  'xxmaj',
  'orleans',
  'accents',
  'in',
  'this',
  'detective',
  'flick',
  '.',
  'xxmaj',
  'it',
  "'s",
  'worth',
  'seeing',
  'for',
  'their',
  'scenes-',
  'and',
  'xxmaj',
  'rickman',
  "'s",
  'scene',
  'with',
  'xxmaj',
  'hal',
  'xxmaj',
  'holbrook',
  '.',
  'xxmaj',
  'these',
  'three',
  'actors',
  'mannage',
  'to',
  'entertain',
  'us',
  'no',
  'matter',
  'what',
  'the',
  'movie',
  ',',
  'it',
  'seems',
  '.',
  'xxmaj',
  'the',
  'plot',
  'for',
  'the',
  'movie',
  'shows',
  'potential',
  ',',
  'but',
  'one',
  'gets',
  'the',
  'impression',
  'in',
  'watching',
  'the',
  'film',
  'that',
  'it',
  'was',
  'not',
  'pulled',
  'off',
  'as',
  'well',
  'as',
  'it',
  'could',
  'have',
  'been',
  '.',
  'xxmaj',
  'the',
  'fact',


In [36]:
#export
import collections

class NumericalizeProcessor(Processor):
    def __init__(self, vocab=None, max_vocab=60000, min_freq=2): 
        self.vocab,self.max_vocab,self.min_freq = vocab,max_vocab,min_freq
    
    def __call__(self, items):
        #The vocab is defined on the first use.
        if self.vocab is None:
            freq = Counter(p for o in items for p in o)
            self.vocab = [o for o,c in freq.most_common(self.max_vocab) if c >= self.min_freq]
            #insert all the special token in the begining of the vocab
            for o in reversed(default_spec_tok):
                if o in self.vocab: self.vocab.remove(o)
                self.vocab.insert(0, o)
        if getattr(self, 'otoi', None) is None:
            self.otoi = collections.defaultdict(int,{v:k for k,v in enumerate(self.vocab)}) 
        return [self.proc1(o) for o in items]
    def proc1(self, item):  return [self.otoi[o] for o in item]
    
    def deprocess(self, idxs):
        assert self.vocab is not None
        return [self.deproc1(idx) for idx in idxs]
    
    def deproc1(self, idx): return [self.vocab[i] for i in idx]

In [37]:
proc_tok,proc_num = TokenizeProcessor(max_workers=8),NumericalizeProcessor()

In [38]:
%time ll = label_by_func(sd, lambda y: 0, proc_x = [proc_tok,proc_num])

CPU times: user 29.1 s, sys: 11.5 s, total: 40.6 s
Wall time: 4min 15s


In [39]:
ll.train[0]

([2,
  7,
  1856,
  7,
  9851,
  203,
  7,
  3612,
  7,
  4374,
  221,
  67,
  379,
  27,
  2420,
  125,
  7,
  181,
  7,
  5610,
  2588,
  17,
  19,
  1201,
  505,
  9,
  7,
  16,
  22,
  294,
  335,
  28,
  80,
  36114,
  11,
  7,
  9851,
  22,
  150,
  27,
  7,
  4245,
  7,
  17775,
  9,
  7,
  152,
  299,
  172,
  0,
  14,
  2828,
  198,
  74,
  518,
  64,
  8,
  29,
  10,
  16,
  209,
  9,
  7,
  8,
  130,
  28,
  8,
  29,
  297,
  1041,
  10,
  30,
  42,
  243,
  8,
  1473,
  17,
  168,
  8,
  31,
  20,
  16,
  25,
  37,
  2008,
  141,
  26,
  89,
  26,
  16,
  95,
  41,
  99,
  9,
  7,
  8,
  212,
  20,
  16,
  15,
  15922,
  47,
  12,
  269,
  2715,
  3334,
  11,
  675,
  2715,
  12056,
  83,
  41993,
  200,
  9,
  7,
  8,
  29,
  15,
  294,
  12,
  0,
  63,
  28,
  183,
  69,
  93,
  438,
  379,
  47,
  7,
  9851,
  10,
  7,
  4374,
  10,
  11,
  7,
  17775,
  9,
  3],
 0)

In [40]:
ll.train.x_obj(0)

"xxbos xxmaj alan xxmaj rickman & xxmaj emma xxmaj thompson give good performances with southern / xxmaj new xxmaj orleans accents in this detective flick . xxmaj it 's worth seeing for their scenes- and xxmaj rickman 's scene with xxmaj hal xxmaj holbrook . xxmaj these three actors xxunk to entertain us no matter what the movie , it seems . xxmaj the plot for the movie shows potential , but one gets the impression in watching the film that it was not pulled off as well as it could have been . xxmaj the fact that it is cluttered by a rather uninteresting subplot and mostly uninteresting kidnappers really muddles things . xxmaj the movie is worth a xxunk if for nothing more than entertaining performances by xxmaj rickman , xxmaj thompson , and xxmaj holbrook . xxeos"

In [41]:
pickle.dump(ll, open(path/'ld.pkl', 'wb'))

In [42]:
ll = pickle.load(open(path/'ld.pkl', 'rb'))

In [43]:
ll

SplitData
Train: LabeledData
x: TextList (90043 items)
[[2, 7, 1856, 7, 9851, 203, 7, 3612, 7, 4374, 221, 67, 379, 27, 2420, 125, 7, 181, 7, 5610, 2588, 17, 19, 1201, 505, 9, 7, 16, 22, 294, 335, 28, 80, 36114, 11, 7, 9851, 22, 150, 27, 7, 4245, 7, 17775, 9, 7, 152, 299, 172, 0, 14, 2828, 198, 74, 518, 64, 8, 29, 10, 16, 209, 9, 7, 8, 130, 28, 8, 29, 297, 1041, 10, 30, 42, 243, 8, 1473, 17, 168, 8, 31, 20, 16, 25, 37, 2008, 141, 26, 89, 26, 16, 95, 41, 99, 9, 7, 8, 212, 20, 16, 15, 15922, 47, 12, 269, 2715, 3334, 11, 675, 2715, 12056, 83, 41993, 200, 9, 7, 8, 29, 15, 294, 12, 0, 63, 28, 183, 69, 93, 438, 379, 47, 7, 9851, 10, 7, 4374, 10, 11, 7, 17775, 9, 3], [2, 18, 41, 131, 19, 29, 11, 18, 87, 37, 475, 28, 19, 29, 6654, 9, 18, 73, 37, 122, 59, 182, 14, 7, 1484, 107, 18, 58, 37, 53, 19, 655, 11, 112, 2011, 5759, 9, 18, 58, 37, 53, 14, 856, 702, 6654, 107, 18, 58, 37, 406, 80, 1079, 9, 7, 154, 73, 18, 159, 14, 7, 2090, 68, 18, 269, 159, 14, 7, 2495, 55, 8, 7, 2214, 7, 4947, 66, 7, 2495

### Language Modelling Databunch: Batching

We have a bit of work to convert our LabelList in a DataBunch as we don't just want batches of IMDB reviews. We want to stream through all the texts concatenated. We also have to prepare the targets that are the newt words in the text. All of this is done with the next object called LM_PreLoader. At the beginning of each epoch, it'll shuffle the articles (if shuffle=True) and create a big stream by concatenating all of them. We divide this big stream in bs smaller streams. That we will read in chunks of bptt length.

In [44]:
#export
# Just using those for illustration purposes, they're not used otherwise.
from IPython.display import display,HTML
import pandas as pd
import numpy as np 

Then if we split it in 6 batches it would give something like this:

In [45]:
stream = """
In this notebook, we will go back over the example of classifying movie reviews we studied in part 1 and dig deeper under the surface. 
First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the Processor used in the data block API.
Then we will study how we build a language model and train it.\n
"""
tokens = np.array(tp([stream])[0])

Then if we have a bptt of 5, we would go over those three batches.

In [46]:
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,\n,xxmaj,in,this,notebook,",",we,will,go,back,over,the,example,of
classifying,movie,reviews,we,studied,in,part,1,and,dig,deeper,under,the,surface,.
\n,xxmaj,first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into
numbers,and,how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have
another,example,of,the,xxmaj,processor,used,in,the,data,block,api,.,\n,xxmaj
then,we,will,study,how,we,build,a,language,model,and,train,it,.,\n\n


Then if we have a bptt of 5, we would go over those three batches.

In [47]:
bs,bptt = 6,5
for k in range(3):
    d_tokens = np.array([tokens[i*seq_len + k*bptt:i*seq_len + (k+1)*bptt] for i in range(bs)])
    df = pd.DataFrame(d_tokens)
    display(HTML(df.to_html(index=False,header=None)))

xxbos,\n,xxmaj,in,this
classifying,movie,reviews,we,studied
\n,xxmaj,first,we,will
numbers,and,how,to,customize
another,example,of,the,xxmaj
then,we,will,study,how


notebook,",",we,will,go
in,part,1,and,dig
look,at,the,processing,steps
it,.,xxmaj,by,doing
processor,used,in,the,data
we,build,a,language,model


back,over,the,example,of
deeper,under,the,surface,.
necessary,to,convert,text,into
this,",",we,'ll,have
block,api,.,\n,xxmaj
and,train,it,.,\n\n


In [48]:
#export
class LM_PreLoader():
    
    def __init__(self, data, bs=64, bptt=70, shuffle=False):
        self.data, self.bs, self.bptt, self.shuffle = data, bs, bptt, shuffle
        total_len = sum([len(x) for x in data.x])
        self.n_batch = total_len // bs
        self.batchify()
        
    def __len__(self): return ((self.n_batch-1) // self.bptt) * self.bs
    
    def __getitem__(self, idx):
        source = self.batched_data[idx % self.bs]
        seq_idx = (idx // self.bs) * self.bptt
        return source[seq_idx:seq_idx+self.bptt],source[seq_idx+1:seq_idx+self.bptt+1]
        
    def batchify(self):
        texts = self.data.x
        if self.shuffle: texts = texts[torch.randperm(len(texts))]
        stream = torch.cat([tensor(t) for t in texts])
        self.batched_data = stream[:self.n_batch*self.bs].view(self.bs, self.n_batch)

In [49]:
dl = DataLoader(LM_PreLoader(ll.valid, shuffle=True), batch_size=64)

In [50]:
iter_dl = iter(dl)
x1,y1 = next(iter_dl)
x2,y2 = next(iter_dl)

In [51]:
x1.size(),y1.size()

(torch.Size([64, 70]), torch.Size([64, 70]))

In [52]:
vocab = proc_num.vocab

In [53]:
" ".join(vocab[o] for o in x1[0])

"xxbos xxmaj pleasantly surprised by xxmaj tyra 's bubbly rendition of a doll 's outlook and reaction to ' real life ' . xxmaj the situation of child angst and need for her mother can be handled seriously or lightly . xxmaj both help get the point across that in tough times , children need love and support . xxmaj so what if it comes from a doll accidentally brought"

In [54]:
" ".join(vocab[o] for o in y1[0])

"xxmaj pleasantly surprised by xxmaj tyra 's bubbly rendition of a doll 's outlook and reaction to ' real life ' . xxmaj the situation of child angst and need for her mother can be handled seriously or lightly . xxmaj both help get the point across that in tough times , children need love and support . xxmaj so what if it comes from a doll accidentally brought to"

In [55]:
" ".join(vocab[o] for o in x2[0])

"to life . xxmaj the movie did not go too overboard on needing to correct the situation as other movies have . xxmaj the doll 's outlook on life based on infusion from commercial ditties and tv perfection helps to show the difference between tv and reality in a way kids can understand . xxmaj talk to you kids about this after the movie but enjoy xxmaj eve 's bubbly"

In [56]:
#export
def get_lm_dls(train_ds, valid_ds, bs, bptt, **kwargs):
    return (DataLoader(LM_PreLoader(train_ds, bs, bptt, shuffle=True), batch_size=bs, **kwargs),
            DataLoader(LM_PreLoader(valid_ds, bs, bptt, shuffle=False), batch_size=2*bs, **kwargs))

def lm_databunchify(sd, bs, bptt, **kwargs):
    return DataBunch(*get_lm_dls(sd.train, sd.valid, bs, bptt, **kwargs))

In [57]:
bs,bptt = 64,70
data = lm_databunchify(ll, bs, bptt)

In [58]:
data

### Text Classification DataBunch: Batching

In [59]:
proc_cat = CategoryProcessor()

In [60]:
il = TextList.from_files(path, include=['train', 'test'])
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='test'))
ll = label_by_func(sd, parent_labeler, proc_x = [proc_tok, proc_num], proc_y=proc_cat)

In [61]:
pickle.dump(ll, open(path/'ll_clas.pkl', 'wb'))

In [62]:
ll = pickle.load(open(path/'ll_clas.pkl', 'rb'))

In [63]:
[(ll.train.x_obj(i), ll.train.y_obj(i)) for i in [1,12552]]

[('xxbos xxmaj well ... tremors i , the original started off in 1990 and i found the movie quite enjoyable to watch . however , they proceeded to make tremors ii and iii . xxmaj trust me , those movies started going downhill right after they finished the first one , i mean , ass blasters ? ? ? xxmaj now , only xxmaj god himself is capable of answering the question " why in xxmaj gods name would they create another one of these dumpster dives of a movie ? " xxmaj tremors iv can not be considered a bad movie , in fact it can not be even considered an epitome of a bad movie , for it lives up to more than that . xxmaj as i attempted to sit though it , i noticed that my eyes started to bleed , and i hoped profusely that the little girl from the ring would crawl through the tv and kill me . did they really think that dressing the people who had stared in the other movies up as though they we \'re from the wild west would make the movie ( with the exact same occurrences ) any better ? honestl

In [64]:
proc_cat.vocab

['neg', 'pos']

In [65]:
#export 
from torch.utils.data import Sampler

For the validation set, we will simply sort the samples by length, and we begin with the longest ones for memory reasons (it's better to always have the biggest tensors first).

In [66]:
#export
class SortSampler(Sampler):
    
    def __init__(self, data_source, key):
        self.data_source, self.key = data_source, key
        
    def __len__(self):
        return len(self.data_source)
    
    def __iter__(self):
        return iter(sorted(list(range(len(self.data_source))), key=self.key, reverse=True))

For the training set, we want some kind of randomness on top of this. So first, we shuffle the texts and build megabatches of size 50 * bs. We sort those megabatches by length before splitting them in 50 minibatches. That way we will have randomized batches of roughly the same length.

Then we make sure to have the biggest batch first and shuffle the order of the other batches. We also make sure the last batch stays at the end because its size is probably lower than batch size.

In [67]:
#export
class SortishSampler(Sampler):
    def __init__(self, data_source, key, bs):
        self.data_source,self.key,self.bs = data_source,key,bs

    def __len__(self) -> int: return len(self.data_source)

    def __iter__(self):
        idxs = torch.randperm(len(self.data_source))
        megabatches = [idxs[i:i+self.bs*50] for i in range(0, len(idxs), self.bs*50)]
        sorted_idx = torch.cat([tensor(sorted(s, key=self.key, reverse=True)) for s in megabatches])
        batches = [sorted_idx[i:i+self.bs] for i in range(0, len(sorted_idx), self.bs)]
        max_idx = torch.argmax(tensor([self.key(ck[0]) for ck in batches]))  # find the chunk with the largest key,
        batches[0],batches[max_idx] = batches[max_idx],batches[0]            # then make sure it goes first.
        batch_idxs = torch.randperm(len(batches)-2)
        sorted_idx = torch.cat([batches[i+1] for i in batch_idxs]) if len(batches) > 1 else LongTensor([])
        sorted_idx = torch.cat([batches[0], sorted_idx, batches[-1]])
        return iter(sorted_idx)

Padding: we had the padding token (that as an id of 1) at the end of each sequence to make them all the same size when batching them. Note that we need padding at the end to be able to use PyTorch convenience functions that will let us ignore that padding (see 12c).

In [68]:
#export
def pad_collate(samples, pad_idx=1, pad_first=False):
    max_len = max([len(s[0]) for s in samples])
    res = torch.zeros(len(samples), max_len).long() + pad_idx
    for i,s in enumerate(samples):
        if pad_first: res[i, -len(s[0]):] = torch.LongTensor(s[0])
        else:         res[i, :len(s[0]) ] = torch.LongTensor(s[0])
    return res, tensor([s[1] for s in samples])

In [69]:
bs = 64
train_sampler = SortishSampler(ll.train.x, key=lambda t: len(ll.train[int(t)][0]), bs=bs)
train_dl = DataLoader(ll.train, batch_size=bs, sampler=train_sampler, collate_fn=pad_collate)

In [70]:
iter_dl = iter(train_dl)
x,y = next(iter_dl)

In [71]:
lengths = []
for i in range(x.size(0)): lengths.append(x.size(1) - (x[i]==1).sum().item())
lengths[:5], lengths[-1]

([3311, 2425, 1621, 1577, 1481], 1001)

In [72]:
x,y = next(iter_dl)
lengths = []
for i in range(x.size(0)): lengths.append(x.size(1) - (x[i]==1).sum().item())
lengths[:5], lengths[-1]

([312, 312, 312, 311, 311], 299)

In [73]:
x

tensor([[   2,   18,   25,  ...,  156,    9,    3],
        [   2,    7, 1522,  ...,   19,   29,    3],
        [   2, 1287, 1061,  ...,  136,    9,    3],
        ...,
        [   2,    7,  106,  ...,    1,    1,    1],
        [   2,   18,   85,  ...,    1,    1,    1],
        [   2,   18,  699,  ...,    1,    1,    1]])

In [74]:
#export
def get_clas_dls(train_ds, valid_ds, bs, **kwargs):
    train_sampler = SortishSampler(train_ds.x, key=lambda t: len(train_ds.x[t]), bs=bs)
    valid_sampler = SortSampler(valid_ds.x, key=lambda t: len(valid_ds.x[t]))
    return (DataLoader(train_ds, batch_size=bs, sampler=train_sampler, collate_fn=pad_collate, **kwargs),
            DataLoader(valid_ds, batch_size=bs*2, sampler=valid_sampler, collate_fn=pad_collate, **kwargs))

def clas_databunchify(sd, bs, **kwargs):
    return DataBunch(*get_clas_dls(sd.train, sd.valid, bs, **kwargs))

In [75]:
bs,bptt = 64,70
data = clas_databunchify(ll, bs)

In [76]:
!python notebook2script.py 05_datablocks_text.ipynb

Converted 05_datablocks_text.ipynb to lib/nb_05.py
